In [1]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.
Found 50947 files belonging to 7 classes.


In [5]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights="imagenet")

In [6]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model.compile(optimizer="Nadam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(training_s, validation_data=validation_s, epochs=3)

Epoch 1/3
12495/12495 [==============================] - 2031s 162ms/step - loss: 1.4126 - accuracy: 0.4568 - val_loss: 1.3191 - val_accuracy: 0.5044
Epoch 2/3
12495/12495 [==============================] - 2021s 162ms/step - loss: 1.3307 - accuracy: 0.4932 - val_loss: 1.2714 - val_accuracy: 0.5216
Epoch 3/3
12495/12495 [==============================] - 1999s 160ms/step - loss: 1.3102 - accuracy: 0.5010 - val_loss: 1.2643 - val_accuracy: 0.5251


In [10]:
for layer in base_model.layers:
    layer.trainable = True

In [11]:
opt = keras.optimizers.Nadam(learning_rate=1e-4)

In [12]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=10, callbacks=[es])

Epoch 1/10
12495/12495 [==============================] - 9365s 747ms/step - loss: 0.7721 - accuracy: 0.7134 - val_loss: 0.5832 - val_accuracy: 0.7907
Epoch 2/10
12495/12495 [==============================] - 9315s 746ms/step - loss: 0.5125 - accuracy: 0.8127 - val_loss: 0.5169 - val_accuracy: 0.8222
Epoch 3/10
12495/12495 [==============================] - 9295s 744ms/step - loss: 0.3834 - accuracy: 0.8610 - val_loss: 0.5345 - val_accuracy: 0.8315
Epoch 4/10
12495/12495 [==============================] - 9276s 742ms/step - loss: 0.3009 - accuracy: 0.8898 - val_loss: 0.5635 - val_accuracy: 0.8327
Epoch 5/10
12495/12495 [==============================] - 9219s 738ms/step - loss: 0.2416 - accuracy: 0.9108 - val_loss: 0.6197 - val_accuracy: 0.8340


In [14]:
model.evaluate(training_s)

12495/12495 [==============================] - 1739s 139ms/step - loss: 0.3318 - accuracy: 0.8796


[0.3317546248435974, 0.8795915246009827]

In [15]:
model.evaluate(validation_s)

1593/1593 [==============================] - 225s 141ms/step - loss: 0.5169 - accuracy: 0.8222


[0.5169474482536316, 0.8222466707229614]

In [16]:
model.save("1001_Effi_nAdam.h5")